# Data Exploration

The notebook explores the NIH Chest X-ray dataset as described below:

>Potential Dataset 2 : NIH Chest X-ray dataset

>Link: https://academictorrents.com/details/e615d3aebce373f1dc8bd9d11064da55bdadede0

>Size: 112,120 frontal-view X-ray images of 30,805 unique patients

>Color Space: Grayscale

>Resolution: 1024 x 1024 pixels

>Downstream task: Classification of common thoracic pathologies include Atelectasis, Consolidation, Infiltration, Pneumothorax, Edema, Emphysema, Fibrosis, Effusion, Pneumonia, Pleural_thickening, Cardiomegaly, Nodule, Mass and Hernia.

>Benchmark: https://arxiv.org/abs/1705.02315

Please note that each image can have a multi-labels. I'm wonder if the multi-task classification is too complicated.





In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import warnings
warnings.filterwarnings("ignore")
from scipy.io import loadmat
from pprint import pprint
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
#from dataloaders.chest_xray import XRayDataSet
import tensorflow as tf
from PIL import Image
from pandas_profiling import ProfileReport

## 1. Import Dataset










In [ ]:
use_cache = False
data_frame_path = "../data/01_raw/Data_Entry_2017.csv"
img_data_path = "../data/01_raw/images-224"
config = dict()
scratch_dir = None
batch_size = 128
buffer_size = 128 * 2

if use_cache:
    train_ds = XRayDataSet(img_data_path, data_frame_path, config=config, scratch_dir=scratch_dir) \
        .prefetch(tf.data.experimental.AUTOTUNE) \
        .batch(batch_size) \
        .cache(cache_dir + "/tf_learn_cache") \
        .shuffle(buffer_size)

else:
    train_ds = XRayDataSet(img_data_path, data_frame_path, config=config, scratch_dir=scratch_dir) \
        .prefetch(tf.data.experimental.AUTOTUNE) \
        .shuffle(buffer_size)\
        .batch(batch_size) 

for (images, labels) in train_ds:
    tf.print(images[0], output_stream=sys.stdout)
    print()
    print("Just printed image. Done!!")
    break

## 2. Visualize X-Ray Samples

In [ ]:
my_img_grid = df['img'].iloc[430].view(1,1,434,636)
for i in range(431,440):
    # my_img_grid[i,None] = df['img'].iloc[i]
    my_img_grid = torch.cat([my_img_grid,df['img'].iloc[i].view(1,1,434,636)])
    print(df['fat'].iloc[i])
    
    print(df['id'].iloc[i])

def imshow(img):
    plt.figure(figsize=(20,20))
    plt.axis("off")
    plt.title("10 Ultrasound images of the same person")
    #img = img / 2 + 0.5     # unnormalize
    
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# show images
imshow(torchvision.utils.make_grid(my_img_grid.type(torch.int), nrow=5))
# print( "The shape of the images is", images.shape)

## 3. Analyze Data Labels

There are 12 features:
1.'Image Index'
2. 'Finding Labels'
3.'Follow-up #'
4. 'Patient ID'
5. 'Patient Age'
6. 'Patient Gender'
7. 'View Position'
8. 'OriginalImage[Width','Height]'
9. 'OriginalImagePixelSpacing[x', 'y]'
10.'Unnamed: 11'


In [ ]:
data_frame_path = "../data/01_raw/Data_Entry_2017.csv"
df = pd.read_csv(data_frame_path) 
print('There are ', len(df), ' labeled images')
df.head()

In [7]:
df['Patient Age'].unique() 

array(['058Y', '081Y', '074Y', '075Y', '076Y', '077Y', '078Y', '079Y',
       '080Y', '082Y', '069Y', '070Y', '073Y', '084Y', '061Y', '060Y',
       '062Y', '056Y', '057Y', '071Y', '066Y', '053Y', '047Y', '048Y',
       '049Y', '063Y', '064Y', '052Y', '068Y', '059Y', '055Y', '072Y',
       '067Y', '046Y', '091Y', '092Y', '087Y', '065Y', '045Y', '054Y',
       '050Y', '051Y', '044Y', '083Y', '033Y', '042Y', '025Y', '031Y',
       '094Y', '089Y', '090Y', '040Y', '085Y', '030Y', '032Y', '034Y',
       '086Y', '037Y', '027Y', '029Y', '036Y', '038Y', '039Y', '043Y',
       '028Y', '041Y', '035Y', '022Y', '023Y', '026Y', '021Y', '088Y',
       '024Y', '017Y', '018Y', '019Y', '020Y', '016Y', '013Y', '014Y',
       '011Y', '012Y', '015Y', '093Y', '009Y', '010Y', '008Y', '006Y',
       '007Y', '004Y', '005Y', '003Y', '002Y', '412Y', '001Y', '414Y',
       '148Y', '011M', '095Y', '150Y', '149Y', '018M', '152Y', '013M',
       '020M', '021M', '151Y', '007M', '019M', '005M', '016M', '023M',
      

In [ ]:
# split labels 
df_labels = pd.concat([df,df['Finding Labels'].str.get_dummies(sep='|')], axis=1)

# 0 for Male and 1 for female
df_labels['Patient Gender'] = df_labels['Patient Gender'].apply(lambda x: 0 if x=='M' else 1)

# 0 for PA and 1 for AP - I need to verify what is this feature
df_labels['View Position'] = df_labels['View Position'].apply(lambda x: 0 if x=='PA' else 1)

# for the patient age, there is an Y in the value
# remove Y from all patient Age
#df_labels['Patient Age'] = df_labels['Patient Age'].apply(lambda x: int(x[0:3]))
df_labels = df_labels.drop(['Finding Labels'], axis=1)

df_labels.head()

In [1]:
plt.figure(figsize=(20,10))
df_labels['Patient ID'].hist(bins =30805)
plt.hlines(df_labels['Patient ID'].count()/)
plt.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-7c9c4f4a32dd>", line 1, in <module>
    plt.figure(figsize=(20,10))
NameError: name 'plt' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_con

TypeError: object of type 'NoneType' has no len()

In [4]:
profile = ProfileReport(df_labels, title='Pandas Profiling Report', explorative=True)

In [5]:
profile.to_file("chest_xray_exploration.html")